## 1. Consolidated files in the unique DataFrame and show the total files extracted

In [1]:
import os
import sys

notebook_path = os.path.dirname(os.path.abspath("data_explorer.ipynb"))
sys.path.append(os.path.dirname(notebook_path))

from etl.utils.common import DefaultOutputFolder as dir
import pandas as pd

files = os.listdir(dir())
dfs = []

if not files: print("No files found in the output folder.")

for file in files:
    if file.endswith(".parquet"):
        df = pd.read_parquet(dir() + file)
        dfs.append(df)
    
allFiles = pd.concat(dfs, ignore_index=True)

# Ordering DataFrame by column name
allFiles = allFiles.sort_values(by=['extracted_at'], ascending=False)

# count the rows in dataframe
allFiles.shape[0]

530

## 1.1 Data set sample, list 5 files

In [2]:
allFiles.head(3)

,code,codein,name,high,low,varBid,pctChange,bid,ask,timestamp,create_date,symbol,extracted_at
31,EUR,SDG,Euro/Libra Sudanesa,624.6036,624.6036,0,0,605.4179,643.7894,1713887707,2024-04-23 12:55:07,EUR-SDG,2024-04-23 15:55:16
510,EUR,SCR,Euro/Rúpias de Seicheles,15.1334,15.1334,0,0,15.1356,15.1312,1713887705,2024-04-23 12:55:05,EUR-SCR,2024-04-23 15:55:16
118,EUR,HTG,Euro/Gourde Haitiano,141.23,141.18,-0.05,-0.04,140.18,142.18,1713880874,2024-04-23 11:01:14,EUR-HTG,2024-04-23 14:01:59


## 2. Change DataTypes and Reorder columns

In [3]:
# Change data types
df = allFiles.astype({'ask': float, 'bid': float, 'varBid': float, 'pctChange': float})

# Show the dataframe
df.head(3)


,code,codein,name,high,low,varBid,pctChange,bid,ask,timestamp,create_date,symbol,extracted_at
31,EUR,SDG,Euro/Libra Sudanesa,624.6036,624.6036,0.00,0.00,605.4179,643.7894,1713887707,2024-04-23 12:55:07,EUR-SDG,2024-04-23 15:55:16
510,EUR,SCR,Euro/Rúpias de Seicheles,15.1334,15.1334,0.00,0.00,15.1356,15.1312,1713887705,2024-04-23 12:55:05,EUR-SCR,2024-04-23 15:55:16
118,EUR,HTG,Euro/Gourde Haitiano,141.23,141.18,-0.05,-0.04,140.1800,142.1800,1713880874,2024-04-23 11:01:14,EUR-HTG,2024-04-23 14:01:59


## 3. Using SQL for Data Exploration
    3.1 What is the currency with the highest ask value?

In [4]:
from pandasql import sqldf

query = """
    SELECT symbol, name, max(ask) max_ask FROM df 
    where code = 'BRL' 
    group by symbol, name
    order by 3 desc limit 1
"""

newDf = sqldf(query, locals())

newDf



,symbol,name,max_ask
0,BRL-LBP,Real Brasileiro/Libra Libanesa,17293.29
